In [1]:
"""
Tech. 1:
    Pick decision tree based approaches as they work better than logistic regession or SVM.
    Random Forest is a gooe method but beware of overfitting.

Tech. 2:
    Upsampling minority class
    
Tech. 3:
    Downsampling majority class
    
Tech. 4:
    A combination of Tech 2 and 3.
    
Tech. 5:
    Penalise learning algorithms that increase cost of classification mistakes on minority classes
    
Tech. 6:
    Generate synthetic data
    
Tech. 7:
    Add appropriate weights to your deep learning model.
    
"""

'\nTech. 1:\n    Pick decision tree based approaches as they work better than logistic regession or SVM.\n    Random Forest is a gooe method but beware of overfitting.\n\nTech. 2:\n    Upsampling minority class\n    \nTech. 3:\n    Downsampling majority class\n    \nTech. 4:\n    A combination of Tech 2 and 3.\n    \nTech. 5:\n    Penalise learning algorithms that increase cost of classification mistakes on minority classes\n    \nTech. 6:\n    Generate synthetic data\n    \nTech. 7:\n    Add appropriate weights to your deep learning model.\n    \n'

In [2]:
import numpy as np
import cv2
import pandas as pd

img = cv2.imread('image/Sandstone_Versa0180_image.tif')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

df = pd.DataFrame()

# Add original pixel values to the data frame as feature 1
img2 = img.reshape(-1)
df['Original_Image'] = img2



In [3]:
# Add other features

# Gabor features
num = 1
kernels = []
for theta in range(2):
    theta = theta/4. * np.pi
    for sigma in (1, 3):
        for lamda in np.arange(0, np.pi, np.pi/4):
            for gamma in (0.05, 0.5):
                #print(theta, sigma, lamda, gamma)
                gabor_label = 'Gabor' + str(num)
                kernel = cv2.getGaborKernel((9, 9), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
                kernels.append(kernel)
                
                fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img
                num += 1
                # print(gabor_label)


In [4]:
labeled_img = cv2.imread('image/Sandstone_Versa0180_mask.png')
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
labeled_img2 = labeled_img.reshape(-1)

df['Labels'] = labeled_img2

print(df.head)

<bound method NDFrame.head of          Original_Image  Gabor1  Gabor2  Gabor3  Gabor4  Gabor5  Gabor6  \
0                     0       0       0       0       0       0       0   
1                     0       0       0       0       0       0       0   
2                     0       0       0       0       0       0       0   
3                     0       0       0       0       0       0       0   
4                     0       0       0       0       0       0       0   
...                 ...     ...     ...     ...     ...     ...     ...   
1019899               0       0       0       0       0       0       0   
1019900               0       0       0       0       0       0       0   
1019901               0       0       0       0       0       0       0   
1019902               0       0       0       0       0       0       0   
1019903               0       0       0       0       0       0       0   

         Gabor7  Gabor8  Gabor9  ...  Gabor24  Gabor25  Gabor26  Gabo

In [5]:
print(df.Labels.unique()) # Look at the labels in our dataframe
print(df['Labels'].value_counts())

# imbalanced data
print(np.unique(df.Labels, return_counts=True))

[ 33 201 231  65]
33     491532
231    439024
201     72927
65      16421
Name: Labels, dtype: int64
(array([ 33,  65, 201, 231], dtype=uint8), array([491532,  16421,  72927, 439024], dtype=int64))


In [6]:
# Dependent variable
Y = df['Labels'].values

X = df.drop(['Labels'],axis=1)

# Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=20)

In [7]:
"""
Tech. 1:
    Pick decision tree based approaches as they work better than logistic regession or SVM.
    Random Forest is a gooe method but beware of overfitting.
    
"""

# import ML algorithm and train the model

from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=10, random_state=42)
model_RF.fit(X_train, y_train)

pred_test = model_RF.predict(X_test)

from sklearn import metrics

print("Acc = ", metrics.accuracy_score(y_test,pred_test))


Acc =  0.9496546246954373


In [8]:
(unique, counts) = np.unique(pred_test, return_counts=True)
print(unique, counts)

[ 33  65 201 231] [196099   6343  27281 178239]


In [9]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred_test)
print(cm)

print("pixel 33 acc = ", cm[0,0] / (cm[0,0]+cm[1,0]+cm[2,0]+cm[3,0]))
print("pixel 65 acc = ", cm[1,1] / (cm[0,1]+cm[1,1]+cm[2,1]+cm[3,1]))
print("pixel 201 acc = ", cm[2,2] / (cm[0,2]+cm[1,2]+cm[2,2]+cm[3,2]))
print("pixel 231 acc = ", cm[3,3] / (cm[0,3]+cm[1,3]+cm[2,3]+cm[3,3]))

[[192179      1   4219    194]
 [     1   6162      2    456]
 [  3836      5  18509   7016]
 [    83    175   4551 170573]]
pixel 33 acc =  0.9800100969408309
pixel 65 acc =  0.9714646066530033
pixel 201 acc =  0.6784575345478538
pixel 231 acc =  0.9569903332042932


In [10]:
# ROC-AUC
from sklearn.metrics import roc_auc_score

prob_test = model_RF.predict_proba(X_test)

print("ROC_AUC score for imbalanced data:")
print(roc_auc_score(y_test, prob_test, multi_class='ovr', labels=[33,65,201,231]))


ROC_AUC score for imbalanced data:
0.9814222268530911


In [11]:
"""
Tech. 2:
    Upsampling minority class

"""

from sklearn.utils import resample
print(df['Labels'].value_counts())

# separate majority and minority classes
df_important = df[df['Labels'] == 201]
df_majority = df.loc[df['Labels'].isin([33, 231])]
df_minority = df[df['Labels'] == 65]

print(df_important, df_majority, df_minority)

33     491532
231    439024
201     72927
65      16421
Name: Labels, dtype: int64
        Original_Image  Gabor1  Gabor2  Gabor3  Gabor4  Gabor5  Gabor6  \
85165               76       0       0     255     205     116      63   
85181               75       0       0     255     197     113      72   
85182               63       0       0     255     176     102      63   
86161              105       0       0     255     250     127      75   
86162              112       0       0     255     255     149      94   
...                ...     ...     ...     ...     ...     ...     ...   
951673              63       0       0     255     209     135      82   
951682              57       0       0     255     159      77      45   
951683              68       0       0     255     202     136      72   
951700              70       0       0     255     255     193     104   
951701              73       0       0     255     253     187      94   

        Gabor7  Gabor8  Gabo

In [12]:
# upsampling minority class and other classes separately
df_important_upsampled = resample(df_important,
                                 replace=True, # samples with replacement
                                 n_samples=400000,
                                 random_state=42)

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=400000,
                                 random_state=42)

# merge data
df_unsampled = pd.concat([df_majority,
                         df_important_upsampled,
                         df_minority_upsampled])
print(df_unsampled['Labels'].value_counts())

33     491532
231    439024
65     400000
201    400000
Name: Labels, dtype: int64


In [13]:
# Dependent variable
Y_upsampled = df_unsampled['Labels'].values

X_upsampled = df_unsampled.drop(['Labels'],axis=1)

# Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_upsampled, Y_upsampled, test_size=0.4, random_state=20)


In [14]:
# import ML algorithm and train the model

from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=10, random_state=42)
model_RF.fit(X_train, y_train)

pred_test = model_RF.predict(X_test)

from sklearn import metrics

print("Acc = ", metrics.accuracy_score(y_test,pred_test))

Acc =  0.9763226590275099


In [15]:
# ROC-AUC
from sklearn.metrics import roc_auc_score

prob_test = model_RF.predict_proba(X_test)

print("ROC_AUC score for imbalanced data:")
print(roc_auc_score(y_test, prob_test, multi_class='ovr', labels=[33,65,201,231]))

ROC_AUC score for imbalanced data:
0.997525670840977


In [16]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred_test)
print(cm)

print("pixel 33 acc = ", cm[0,0] / (cm[0,0]+cm[1,0]+cm[2,0]+cm[3,0]))
print("pixel 65 acc = ", cm[1,1] / (cm[0,1]+cm[1,1]+cm[2,1]+cm[3,1]))
print("pixel 201 acc = ", cm[2,2] / (cm[0,2]+cm[1,2]+cm[2,2]+cm[3,2]))
print("pixel 231 acc = ", cm[3,3] / (cm[0,3]+cm[1,3]+cm[2,3]+cm[3,3]))

[[190373      1   6068     91]
 [     0 159723      0      0]
 [   379      1 158987    916]
 [    20    425   8489 166750]]
pixel 33 acc =  0.9979084981024469
pixel 65 acc =  0.9973337496097409
pixel 201 acc =  0.9161192550592357
pixel 231 acc =  0.9939972698605721


In [17]:
"""
Tech. 3:
    Downsampling majority class
"""

df_majority_downsampled = resample(df_majority, replace=False, n_samples=30000, random_state=42)

# merge data
df_balanced = pd.concat([df_majority_downsampled,
                         df_important,
                         df_minority])
print(df_balanced['Labels'].value_counts())

201    72927
65     16421
33     15869
231    14131
Name: Labels, dtype: int64


In [18]:
df_majority_downsampled

,Original_Image,Gabor1,Gabor2,Gabor3,Gabor4,Gabor5,Gabor6,Gabor7,Gabor8,Gabor9,...,Gabor24,Gabor25,Gabor26,Gabor27,Gabor28,Gabor29,Gabor30,Gabor31,Gabor32,Labels
867675,17,0,0,117,76,47,33,25,18,0,...,0,0,0,50,27,31,29,0,0,33
742687,122,0,0,255,255,172,105,57,42,0,...,51,0,0,255,255,70,38,87,46,231
418120,130,0,0,255,255,233,134,108,66,0,...,51,0,0,255,255,50,20,122,77,231
386303,41,0,0,218,125,69,42,26,16,0,...,19,0,0,255,255,9,0,49,31,33
342794,122,0,0,255,255,212,126,87,57,0,...,38,0,0,255,255,60,29,122,73,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789366,119,0,0,255,255,193,103,78,39,0,...,55,0,0,255,255,94,56,138,77,231
961428,0,0,0,0,0,0,0,0,0,0,...,1,0,0,39,27,0,0,19,14,33
342923,116,0,0,255,255,212,116,97,53,0,...,46,0,0,255,255,51,26,118,74,231
203858,112,0,0,255,255,185,93,70,31,0,...,28,0,0,255,255,60,33,79,40,231


In [19]:
"""
Tech. 5:
    Penalise learning algorithms that increase cost of classification mistakes on minority classes
"""
# import ML algorithm and train the model

from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=10,
                                  class_weight='balanced',
                                  random_state=42)
model_RF.fit(X_train, y_train)

pred_test = model_RF.predict(X_test)

from sklearn import metrics

print("Acc = ", metrics.accuracy_score(y_test,pred_test))

# ROC-AUC
from sklearn.metrics import roc_auc_score

prob_test = model_RF.predict_proba(X_test)

print("ROC_AUC score for imbalanced data:")
print(roc_auc_score(y_test, prob_test, multi_class='ovr', labels=[33,65,201,231]))

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred_test)
print(cm)

print("pixel 33 acc = ", cm[0,0] / (cm[0,0]+cm[1,0]+cm[2,0]+cm[3,0]))
print("pixel 65 acc = ", cm[1,1] / (cm[0,1]+cm[1,1]+cm[2,1]+cm[3,1]))
print("pixel 201 acc = ", cm[2,2] / (cm[0,2]+cm[1,2]+cm[2,2]+cm[3,2]))
print("pixel 231 acc = ", cm[3,3] / (cm[0,3]+cm[1,3]+cm[2,3]+cm[3,3]))

Acc =  0.9764483410692797
ROC_AUC score for imbalanced data:
0.9975076610899076
[[190454      3   5981     95]
 [     0 159723      0      0]
 [   377      1 158963    942]
 [    19    433   8452 166780]]
pixel 33 acc =  0.9979250720461095
pixel 65 acc =  0.9972714785214786
pixel 201 acc =  0.9167627857620706
pixel 231 acc =  0.9938206498745658
